In [1]:
# STEP 1: Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
%cd /content/drive/MyDrive/OptiML/repo/OptML-project

/content/drive/MyDrive/OptiML/repo/OptML-project


In [ ]:
import os
import numpy as np
import torch
import torch.nn as nn
import torchvision.transforms as transforms
from torchvision.datasets import CIFAR10
from torch.utils.data import DataLoader, TensorDataset
import pandas as pd
from tqdm import tqdm
from sklearn.metrics import f1_score
from Functions.implementations import *

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/IPython/core/interactiveshell.py", line 3553, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-3-e8682c557a04>", line 3, in <cell line: 0>
    import torch
  File "/usr/local/lib/python3.11/dist-packages/torch/__init__.py", line 2108, in <module>
    from torch import _VF as _VF, functional as functional  # usort: skip
    ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/torch/functional.py", line 7, in <module>
    import torch.nn.functional as F
  File "/usr/local/lib/python3.11/dist-packages/torch/nn/__init__.py", line 8, in <module>
    from torch.nn.modules import *  # usort: skip # noqa: F403
    ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/torch/nn/modules/__init__.py", line 1, in <module>
    from .module import Module  # usort: skip
    ^^^^^^^^^^^^^^^^^^^^^^^^^^
  File

In [ ]:
#if google drive :
drive_base_path = '/content/drive/MyDrive/OptiML/repo'
cifar10_c_path = os.path.join(drive_base_path, 'CIFAR-10-C')
model_path = os.path.join(drive_base_path, 'OptML-project/Results/SGD/ResNet_Transform_lr_0.05_momentum_0.9.pth')

In [ ]:
# STEP 4: Load CIFAR-10 test labels
# Load the official CIFAR-10 test set (same order as CIFAR-10-C labels)
test_set = CIFAR10(root='./data', train=False, download=True)
true_labels = torch.tensor(test_set.targets)  # Should have 10,000 labels


In [ ]:
# STEP 5: Load your model
# Replace with your actual model class
from Functions.implementations import *

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = get_resnet18_cifar().to(device)
model.load_state_dict(torch.load(model_path, map_location=device))
model.eval()

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): Identity()
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), p

In [ ]:
# STEP 6: Evaluation function
def evaluate_model_on_corruption(corruption_type, severity):
    corruption_file = os.path.join(cifar10_c_path, f"{corruption_type}.npy")
    data = np.load(corruption_file)[(severity - 1) * 10000: severity * 10000]
    data = torch.tensor(data).permute(0, 3, 1, 2).float() / 255.0  # Normalize to [0,1]
    mean = torch.tensor([0.4914, 0.4822, 0.4465]).view(1, 3, 1, 1)
    std = torch.tensor([0.2023, 0.1994, 0.2010]).view(1, 3, 1, 1)

    data = (data - mean) / std

    dataset = TensorDataset(data, true_labels)
    loader = DataLoader(dataset, batch_size=128, shuffle=False)

    all_preds = []
    all_labels = []

    with torch.no_grad():
        for images, labels in loader:
            images = images.to(device)
            outputs = model(images)
            _, predicted = outputs.max(1)

            all_preds.extend(predicted.cpu().numpy())
            all_labels.extend(labels.numpy())

    f1 = f1_score(all_labels, all_preds, average='macro')  # You can also use 'weighted' if you prefer
    return f1

In [ ]:
# STEP 7: Run evaluation on all corruptions and severity levels
corruptions = [
    "gaussian_noise", "shot_noise", "impulse_noise",
    "defocus_blur", "glass_blur", "motion_blur", "zoom_blur",
    "snow", "frost", "fog", "brightness",
    "contrast", "elastic_transform", "pixelate", "jpeg_compression"
]

results = []

for corruption in tqdm(corruptions):
    for severity in range(1, 6):
        f1 = evaluate_model_on_corruption(corruption, severity)
        results.append({
          'corruption': corruption,
          'severity': severity,
          'f1_macro': f1
        })

100%|██████████| 15/15 [05:44<00:00, 22.99s/it]


In [ ]:
# STEP 8: Save results to CSV
df = pd.DataFrame(results)
csv_path = os.path.join(drive_base_path, 'ResNet_corruption_evaluation_results.csv')
df.to_csv(csv_path, index=False)

df.head()


,corruption,severity,f1_macro
0,gaussian_noise,1,0.794151
1,gaussian_noise,2,0.646899
2,gaussian_noise,3,0.478483
3,gaussian_noise,4,0.409692
4,gaussian_noise,5,0.333024


In [ ]:
def evaluate_on_clean_testset():
    transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.4914, 0.4822, 0.4465],
                         std=[0.2023, 0.1994, 0.2010])
    ])
    test_dataset = CIFAR10(root='./data', train=False, download=True, transform=transform)
    test_loader = DataLoader(test_dataset, batch_size=128, shuffle=False)

    all_preds = []
    all_labels = []

    model.eval()
    with torch.no_grad():
        for images, labels in test_loader:
            images = images.to(device)
            outputs = model(images)
            _, predicted = outputs.max(1)

            all_preds.extend(predicted.cpu().numpy())
            all_labels.extend(labels.numpy())

    f1 = f1_score(all_labels, all_preds, average='macro')
    return f1

# Call the function and print the result
clean_f1 = evaluate_on_clean_testset()
print(f"F1-score on clean CIFAR-10 test set: {clean_f1:.4f}")

F1-score on clean CIFAR-10 test set: 0.9193
